
# Trabajo práctico integrador

Nombre: _Diego Gutierrez_

## Primera parte (clases 1 y 2)

In [83]:
import numpy as np
import matplotlib.pyplot as plt

### Primer ejercicio

Dada una matriz en formato *numpy array*, donde cada fila de la matriz representa un vector matemático, se requiere computar las normas $l_0$, $l_1$, $l_2$, $l_{\infty}$, según la siguientes definiciones:

\begin{equation}
    ||\mathbf{x}||^{p} = \bigg(\sum_{j=1}^{n}{|x_i|^p}\bigg)^{\frac{1}{p}}
\end{equation}

con los casos especiales para $p=0$ y $p=\infty$ siendo:

\begin{equation}
    \begin{array}{rcl}
        ||\mathbf{x}||_0 & = & \bigg(\sum_{j=1 \wedge x_j != 0}{|x_i|}\bigg)\\
        ||\mathbf{x}||_{\infty} & = & \max_{i}{|x_i|}\\
    \end{array}
\end{equation}

In [84]:
# Las funciones están implementadas para calcular la norma de cada vector individual x,
# no de la matriz completa

# norma para caso especial 0
def norma0(x):
    # version ayudada por numpy
    #return np.sum(np.abs(x) != 0)
    # version manual
    q = 0
    for xi in x:
        if abs(xi) != 0:
            q += 1
    return q

# norma para caso especial infinito
def normainf(x):
    # version ayudada por numpy
    #return np.max(np.abs(x))
    # version manual
    m = 0
    for xi in x:
        if abs(xi) > m:
            m = abs(xi)
    return m

# norma vector
def normavec(x, p):
    if p == np.inf:
        return normainf(x)
    elif p == 0:
        return norma0(x)
    else:
        # version ayudada por numpy
        #return np.sum(np.abs(x) ** p) ** (1 / p)
        # version manual
        s = 0
        for xi in x:
            s += abs(xi) ** p
        return s ** (1 / p)

In [85]:
A = np.array([
    [-4, 0, 7, -9, 3, 0, 6],
    [-1.5, 2.5, 0, 0, 7.3, -9.9, 1]
])
for x in A:
    print(f"vector: {x}")
    for i in [0, 1, 2, np.inf]:
        print(f"norma {i}: {normavec(x, i)}")
    print()

vector: [-4.  0.  7. -9.  3.  0.  6.]
norma 0: 5
norma 1: 29.0
norma 2: 13.820274961085254
norma inf: 9.0

vector: [-1.5  2.5  0.   0.   7.3 -9.9  1. ]
norma 0: 5
norma 1: 22.200000000000003
norma 2: 12.680693987317888
norma inf: 9.9



### Segundo Ejercicio

En clasificación contamos con dos arreglos, la “verdad” y la “predicción”. Cada elemento de los arreglos pueden tomar dos valores, “True” (representado por 1) y “False” (representado por 0). Entonces podemos definir 4 variables:

* True Positive (TP): El valor verdadero es 1 y el valor predicho es 1
* True Negative (TN): El valor verdadero es 0 y el valor predicho es 0
* False Positive (FP): El valor verdadero es 0 y el valor predicho es 1
* False Negative (FN): El valor verdadero es 1 y el valor predicho es 0

A partir de esto definimos:

* Precision = TP / (TP + FP)
* Recall = TP / (TP + FN)
* Accuracy = (TP + TN) / (TP + TN + FP + FN)
 
Calcular las 3 métricas con Numpy y operaciones vectorizadas.

In [86]:
truth      = np.array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1])
prediction = np.array([1, 1, 1, 1, 0, 0, 1, 1, 0, 0])

In [87]:
# La funcion usa variables globales, que es feo pero la mantiene simple de leer
def cls_case(tr, pred):
    return np.sum(np.logical_and(truth == tr, prediction == pred))

def classif_precision():
    tp = cls_case(1, 1)
    fp = cls_case(0, 1)
    if tp + fp > 0:
        return tp / (tp + fp)
    else:
        return None

def classif_recall():
    tp = cls_case(1, 1)
    fn = cls_case(1, 0)
    if tp + fn > 0:
        return tp / (tp + fn)
    else:
        return None

def classif_accuracy():
    tp = cls_case(1, 1)
    tn = cls_case(0, 0)
    fp = cls_case(0, 1)
    fn = cls_case(1, 0)
    if tp + tn + fp + fn > 0:
        return (tp + tn) / (tp + tn + fp + fn)
    else:
        return None

In [88]:
print(f"precision: {classif_precision()}")
print(f"recall: {classif_recall()}")
print(f"accuracy: {classif_accuracy()}")

precision: 0.5
recall: 0.5
accuracy: 0.4


### Tercer ejercicio

Crear una función que separe los datos en train-validation-test. Debe recibir de parametros:

- X: Array o Dataframe que contiene los datos de entrada del sistema.
- y: Array o Dataframe que contiene la(s) variable(s) target del problema.
- train_percentage: _float_ el porcentaje de training.
- test_percentage: _float_ el porcentaje de testing.
- val_percentage: _float_ el porcentaje de validación.
- shuffle: _bool_ determina si el split debe hacerse de manera random o no.

Hints: 

* Usar Indexing y slicing
* Usar np.random.[...]

In [89]:
def split(X_input,
          Y_input,
          train_size=0.7,
          val_size=0.15,
          test_size=0.15,
          random_state=42,
          shuffle=True):

      # Ver que las proporciones sumen 100%
      if train_size + val_size + test_size != 1.0:
            raise ArithmeticError("train + test + val deben sumar 100%")

      # Confirmar que los arrays se correspondan
      if X_input.shape[0] != Y_input.shape[0]:
            raise TypeError("X e y deben tener la misma dimension principal")

      # Unir X e Y en un unico array para simplificar el manejo,
      # poniendo Y como primera columna
      dataset = np.insert(X_input, 0, Y_input, axis=1)

      # Inicializar random
      rng = np.random.default_rng(random_state)

      # Mezclar dataset si corresponde
      if shuffle:
            rng.shuffle(dataset)

      # Calcular cantidad de registros, redondeando como sea necesario
      q = dataset.shape[0]
      q_train = round(q * train_size)
      q_val = round(q * val_size)

      # Conjuntos de salida
      tmp_train = dataset[0:q_train, :]
      tmp_val = dataset[q_train:(q_train + q_val), :]
      tmp_test = dataset[(q_train + q_val):, :]

      return {
            'train': [tmp_train[:, 1:], tmp_train[:, 0]],
            'val': [tmp_val[:, 1:], tmp_val[:, 0]],
            'test': [tmp_test[:, 1:], tmp_test[:, 0]]
      }

In [90]:
# Datos de prueba usados en otra materia, ideales para probar la funcion split()
from sklearn.datasets import load_iris

def get_iris_dataset():
  data = load_iris()
  return data.data, data.target

data, target = get_iris_dataset()

partitioned = split(data, target, train_size=.65, val_size=.15, test_size=.2)
partitioned

{'train': [array([[5.9, 3.2, 4.8, 1.8],
         [5.7, 2.6, 3.5, 1. ],
         [6.1, 2.6, 5.6, 1.4],
         [4.6, 3.1, 1.5, 0.2],
         [6.3, 2.7, 4.9, 1.8],
         [6.9, 3.2, 5.7, 2.3],
         [5.5, 2.4, 3.7, 1. ],
         [7.6, 3. , 6.6, 2.1],
         [7.4, 2.8, 6.1, 1.9],
         [5.5, 4.2, 1.4, 0.2],
         [6.7, 3.3, 5.7, 2.1],
         [5. , 3.6, 1.4, 0.2],
         [4.8, 3.4, 1.9, 0.2],
         [6.4, 2.7, 5.3, 1.9],
         [7.2, 3. , 5.8, 1.6],
         [5. , 3. , 1.6, 0.2],
         [4.7, 3.2, 1.3, 0.2],
         [5.4, 3.4, 1.7, 0.2],
         [7.2, 3.2, 6. , 1.8],
         [6. , 3.4, 4.5, 1.6],
         [4.9, 3.6, 1.4, 0.1],
         [6.8, 3.2, 5.9, 2.3],
         [6.2, 2.8, 4.8, 1.8],
         [6.1, 3. , 4.6, 1.4],
         [5.1, 3.5, 1.4, 0.2],
         [6.3, 2.3, 4.4, 1.3],
         [6. , 2.9, 4.5, 1.5],
         [5.2, 3.5, 1.5, 0.2],
         [5.8, 2.7, 3.9, 1.2],
         [5.5, 2.6, 4.4, 1.2],
         [5.6, 2.7, 4.2, 1.3],
         [6.3, 3.3, 4.7, 1.6],